In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression,Lasso

In [2]:
df=pd.read_csv('training_data.csv')

In [3]:
# def preprocess()

x=df.drop(['y'],axis=1).values
y=df['y'].values#.reshape(-1,1)

instances,predictors=x.shape

In [4]:
# def preprocess()

x=df.drop(['y'],axis=1).values
y=df['y'].values#.reshape(-1,1)

instances,predictors=x.shape

x=np.concatenate([x,np.ones(shape=(x.shape[0],1))],axis=1)

#We have an alzebraic formulation for all the predictors and intercept, we need to incorporate z based pivot variables in our loss function
#This needs to be done beacus eof the design of scipy optimization instance that requires variable to be of fixed length in our constraint and loss function

expanded_loss_term1=np.zeros(shape=(2*predictors+1,2*predictors+1))
expanded_loss_term1[:predictors+1,:predictors+1]=x.T@x


expanded_loss_term2=np.zeros(
    shape=(2*predictors+1)
)
expanded_loss_term2[:predictors+1]=-2*(y.T@x)

#Thus we have formulated two terms for our loss function

def loss_fn(vars):
  t_1=vars.T@expanded_loss_term1@vars
  t_2=expanded_loss_term2@vars
  return t_1 + t_2



In [5]:
#Finding appropriate M by computing regression coefficients of our dataset

lt_coefs=[]

lr=LinearRegression()
lr.fit(x,y)
lt_coefs.extend(lr.coef_)

for i in range(x.shape[1]):
  lr=LinearRegression()
  lr.fit(x[:,i].reshape(-1,1),y)
  lt_coefs.extend(lr.coef_)


print(np.max(np.absolute(np.array(lt_coefs))))
# Our starting M can be--> 100 >>2.8*10
M=np.round(np.max(np.absolute(np.array(lt_coefs)))*10,-2)
M=20

bounds=[(0,1.1) for _ in range(51)]+[(-M,M) for _ in range(predictors)]


2.8017076418520306


In [6]:
k=5#Variable allowed
def num_vars(vars):
  sum_booleans=np.sum(vars[predictors+1:])
  return k-sum_booleans

constr_1={'type':'eq','fun':num_vars}

lt_constraints=[constr_1]

In [7]:
from functools import partial

def upper_const(vars,ii):
  return vars[ii] - M*vars[predictors+1+ii]

dict_fn_upper={f'fn_{i}': partial(upper_const,ii=i) for i in range(predictors)}

def lower_const(vars,ii):
  return -vars[ii] + M*vars[predictors+1+ii]

dict_fn_lower={f'fn_{i}': partial(lower_const,ii=i) for i in range(predictors)}

#M BOunding constraints
for i in range(predictors):
  temp_constr_1={
      'type':'ineq',
      'fun' : dict_fn_upper[f'fn_{i}']
  }
  lt_constraints.append(temp_constr_1)

  temp_constr_2={
      'type':'ineq',
      'fun' : dict_fn_lower[f'fn_{i}']
  }
  lt_constraints.append(temp_constr_2)



# Binary constraints

def binary_const(vars,ii):
  return vars[ii]*(1-vars[ii])

dict_fn={f'fn_{i}': partial(binary_const,ii=i) for i in range(predictors+1,2*predictors+1)}

for i in range(predictors+1,2*predictors+1):
    temp_constr={'type':'eq',
                 'fun': dict_fn[f'fn_{i}']}
    lt_constraints.append(temp_constr)



In [8]:
# len(lt_constraints)
# # lt_constraints[100]
# temp_lt=list(range(101))
# # i=2
# # temp_lt
# dict_fn['fn_55'](temp_lt)
# lt_constraints[2]['fun'](temp_lt)

In [9]:
lt_coefs=[]

lr=LinearRegression()
lr.fit(x,y)
lt_coefs.extend(lr.coef_)
initial_vals=lt_coefs+[0]*50

In [10]:
from scipy.optimize import minimize
beta_vals = minimize(loss_fn,initial_vals,
                     bounds=bounds,constraints=lt_constraints)

In [11]:
np.sum(beta_vals.x[predictors+1:])

4.981689453125

In [21]:
len(lt_constraints)
beta_vals.x[predictors+1:]

array([ 3.46679687e-02,  5.14404297e-01,  5.61523438e-03,  2.49755859e-01,
        8.54248047e-01,  3.54248047e-01,  1.92871094e-02,  4.84130859e-01,
       -7.08007812e-03,  8.56201172e-01,  1.79443359e-01,  4.80224609e-01,
        7.00683594e-02,  4.95605469e-02, -1.19628906e-02,  4.46777344e-02,
        1.92871094e-02, -1.00097656e-02,  9.54589844e-02,  2.70996094e-02,
        7.56835938e-03,  1.85302734e-01,  4.76074219e-02,  6.51855469e-02,
       -7.08007813e-03,  8.66699219e-02,  6.59179687e-03,  3.78417969e-02,
        8.27636719e-02, -3.17382813e-03,  8.47167969e-02,  7.32421875e-04,
       -1.87988281e-02,  8.54492188e-03, -1.22070312e-03,  3.97949219e-02,
        8.54492187e-03,  1.24511719e-02,  1.24511719e-02,  1.24511719e-02,
       -3.17382812e-03,  7.32421875e-04,  1.24511719e-02,  8.54492188e-03,
        4.63867187e-03, -7.08007812e-03,  7.32421875e-04, -7.08007812e-03,
       -3.17382813e-03, -3.17382812e-03])

In [25]:
lt_constraints[104]['fun'](beta_vals.x)

0.187377870082855

In [13]:
# bb=np.random.normal(size=(2*predictors+1))
# expanded_loss_term2@bb
# bb.T@expanded_loss_term1@bb


lt_f=[]
delta_t=0
for i in range(3):
  def fn(a):
    # global delta_t
    return (a + delta_t)
  fn.__name__='fn_'+str(delta_t)

  delta_t+=1
  lt_f.append(fn)

In [14]:
[i.__name__ for i in lt_f]

['fn_0', 'fn_1', 'fn_2']

In [15]:
def wrapper(f):
  def inner(*args):
    print('wassup')
    return f(*args)
  return inner



In [16]:
def fn(a):
  return a+1


fn=wrapper(fn)
# fn(10)

@wrapper
def fn0(a):
  return a*1

fn0(0.1)

wassup


0.1

In [17]:
# https://stackoverflow.com/questions/15331726/how-does-functools-partial-do-what-it-does

def wrapper(f, *add_args):
  def inner(*args):
    lt_args=list(args)
    lt_args.extend(add_args)#this arguments will be thus added after so see the positional argument of the parent function to 
    
    return f(*lt_args)
  return inner

In [18]:
bb=np.arange(5)
bb

array([0, 1, 2, 3, 4])

In [19]:
def constraint(M,i):
  global bb
  return M - bb[i] 

lt_fn=[]
# const=wrapper(const,)
lt_fn=[wrapper(constraint,i) for i in range(len(bb))]

In [20]:
[f(100) for f in lt_fn]#This will execute like inner(100) inner(100) inner(100) inner(100) inner(100)__> constraint(100,0),constraint(100,1),constraint(100,2),constraint(100,3)constraint(100,4)

[100, 99, 98, 97, 96]